<a href="https://colab.research.google.com/github/Traders-who-code-labs/position-size-calculator/blob/main/technical_analysis_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [32]:
import yfinance as yf
import streamlit as st # Streamlit is now installed and can be imported
import datetime
import pandas as pd
import cufflinks as cf
from plotly.offline import iplot
cf.go_offline()

In [35]:
@st.cache
def get_sp500_components():
    df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    df = df[0]
    # Print the DataFrame to inspect the column names
    print(df.columns)
    # Assuming the column containing tickers is now named 'Symbol'
    tickers = df['Symbol'].to_list()
    tickers_companies_dict = dict(
        zip(df["Symbol"], df["Security"])
    )
    return tickers, tickers_companies_dict

2024-07-29 21:50:05.955 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-07-29 21:50:05.958 No runtime found, using MemoryCacheStorageManager


In [36]:
@st.cache
def load_data(symbol, start, end):
 return yf.download(symbol, start, end)

2024-07-29 21:50:35.949 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-07-29 21:50:35.952 No runtime found, using MemoryCacheStorageManager


In [38]:
@st.cache
def convert_df_to_csv(df):
 return df.to_csv().encode("utf-8")

2024-07-29 21:51:03.060 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-07-29 21:51:03.064 No runtime found, using MemoryCacheStorageManager


In [43]:
st.sidebar.header("Stock Parameters")
available_tickers, tickers_companies_dict = get_sp500_components()
ticker = st.sidebar.selectbox(
"Ticker",
available_tickers,
format_func=tickers_companies_dict.get
)
start_date = st.sidebar.date_input(
"Start date",
datetime.date(2019, 1, 1))
end_date = st.sidebar.date_input(
"End date",
datetime.date.today()
)
if start_date > end_date:
    st.sidebar.error("The end date must fall after the start date")

2024-07-29 21:52:26.022 No runtime found, using MemoryCacheStorageManager
2024-07-29 21:52:26.386 Session state does not function when running a script without `streamlit run`


Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')


In [44]:
st.sidebar.header("Technical Analysis Parameters")
volume_flag = st.sidebar.checkbox(label="Add volume")

In [48]:
exp_sma = st.sidebar.expander("SMA")
sma_flag = exp_sma.checkbox(label="Add SMA")
sma_periods= exp_sma.number_input(
label="SMA Periods",
min_value=1,
)
max_value=50,
value=20,
step=1

In [51]:
exp_bb = st.sidebar.expander("Bollinger Bands")
bb_flag = exp_bb.checkbox(label="Add Bollinger Bands")
bb_periods= exp_bb.number_input(label="BB Periods",
min_value=1,
max_value=50,
value=20, step=1)
bb_std= exp_bb.number_input(label="# of standard deviations",
min_value=1, max_value=4, value=2, step=1)

In [52]:
exp_rsi = st.sidebar.expander("Relative Strength Index")
rsi_flag = exp_rsi.checkbox(label="Add RSI")
rsi_periods= exp_rsi.number_input(
label="RSI Periods",
min_value=1,
max_value=50,
value=20,
step=1
)
rsi_upper= exp_rsi.number_input(label="RSI Upper",
min_value=50,
max_value=90, value=70,
step=1)
rsi_lower= exp_rsi.number_input(label="RSI Lower",
min_value=10,
max_value=50, value=30,
step=1)

In [53]:
st.title("A simple web app for technical analysis")
st.write("""### User manual
* you can select any company from the S&P 500
constituents
""")

In [54]:
df = load_data(ticker, start_date, end_date)

2024-07-29 21:55:11.786 No runtime found, using MemoryCacheStorageManager
[*********************100%%**********************]  1 of 1 completed


In [55]:
data_exp = st.expander("Preview data")
available_cols = df.columns.tolist()
columns_to_show = data_exp.multiselect(
"Columns",
available_cols,
default=available_cols
)
data_exp.dataframe(df[columns_to_show])
csv_file = convert_df_to_csv(df[columns_to_show])
data_exp.download_button(
label="Download selected as CSV",
data=csv_file,
file_name=f"{ticker}_stock_prices.csv",
mime="text/csv",
)

2024-07-29 21:55:28.217 No runtime found, using MemoryCacheStorageManager


False

In [58]:
title_str = f"{tickers_companies_dict[ticker]}'s stock price"
qf = cf.QuantFig(df, title=title_str)
if volume_flag:
    qf.add_volume()  # Indent this line
if sma_flag:
    qf.add_sma(periods=sma_periods)
if bb_flag:
    qf.add_bollinger_bands(periods=bb_periods,
                           boll_std=bb_std)
if rsi_flag:
    qf.add_rsi(periods=rsi_periods,
               rsi_upper=rsi_upper,
               rsi_lower=rsi_lower,
               showbands=True)
fig = qf.iplot(asFigure=True)
st.plotly_chart(fig)

DeltaGenerator()

In [61]:
!streamlit run technical_analysis_app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: technical_analysis_app.py
